In [1]:
# require at least Python 3.5 for async/await to work
import sys
ver=sys.version_info
assert (ver[0]>=3 and ver[1]>=5)
sys.path.append('../../quant_container/src')
sys.path.append('../src')
from aiostreams import run
import aiostreams.operators as op
from mosaicsmartdata.common import qc_csv_helper
from mosaicsmartdata.core.quote import Quote
import csv


In [2]:
filename = '../resources/fx/EUR_JPY_GBP_till_SN.csv'
quote_dict = qc_csv_helper.get_fx_quotes(filename)
#         i+=1
#         if i > 5:
#             break

100000 <class 'mosaicsmartdata.core.quote.Quote'> :  duration = None, ask = 1.119, bid = 1.1189, sym = EUR=, timestamp = 2017-06-27 03:11:19.399000, instr = <class 'mosaicsmartdata.core.instrument.FXForward'> :  tenor = SPOT, ccy = ('EUR', 'USD'), notionals = None, venue = None, static = Instrument static singleton facade, holidayCities = None, sym = EUR=, settle_date = 2017-06-27, isDeliverable = True, date_calc = <mosaicsmartdata.core.date_calculator.DateCalculator object at 0x0000027C35615860>
200000 <class 'mosaicsmartdata.core.quote.Quote'> :  duration = None, ask = 1.1348, bid = 1.1345, sym = EUR=, timestamp = 2017-06-27 20:16:40.312000, instr = <class 'mosaicsmartdata.core.instrument.FXForward'> :  tenor = SPOT, ccy = ('EUR', 'USD'), notionals = None, venue = None, static = Instrument static singleton facade, holidayCities = None, sym = EUR=, settle_date = 2017-06-27, isDeliverable = True, date_calc = <mosaicsmartdata.core.date_calculator.DateCalculator object at 0x0000027C35615

In [5]:
spot_quotes = [quotes for sym, quotes in quote_dict.items() if len(sym)==4]
len(spot_quotes)

3